# First Assignment

In [1]:
import pandas as pd # library for data analsysis
!pip install lxml

## Create a dataframe

In [2]:
# Extract table from wikipedia
from pandas.io.html import read_html
page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table=read_html(page,attrs={"class":"wikitable"})
print('Table extracted')

Table extracted


In [3]:
# Check the first rows of the dataframe
pc_table=table[0]
pc_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# Rename the 1st column
pc_table.rename(columns={'Postcode':'PostalCode'},inplace=True)
pc_table.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Remove rows with 'Not assigned' borough

In [5]:
filtered_pc_table=pc_table[pc_table['Borough']!='Not assigned'].reset_index(drop=True)
filtered_pc_table.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## Check if 'Not assigned' neighborhood

In [6]:
n=filtered_pc_table[filtered_pc_table['Neighbourhood']=='Not assigned']
print('There are {} neighbourhood not assigned.'.format(n.shape[0]))

There are 0 neighbourhood not assigned.


## Combine rows to get a table with unique Postal Codes

In [7]:
PostalCodes_table = filtered_pc_table.groupby(['PostalCode','Borough'],as_index=False,sort=False).agg(lambda x:', '.join(x))
PostalCodes_table.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [8]:
print('There are {} rows in the PostalCodes_table.'.format(PostalCodes_table.shape[0]))

There are 103 rows in the PostalCodes_table.


# Second Assignment

### Unable to obtain all the coordinates with geocoder, so use the csv file

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded')

Data downloaded


In [10]:
latlon=pd.read_csv('Geospatial_Coordinates.csv')
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge the dataframes

In [14]:
latlon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
PC=pd.merge(PostalCodes_table,latlon,on='PostalCode')
PC.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
